In [7]:
import numpy as np
import pandas as pd

In [8]:
movie_df = pd.read_csv('./df_movies_final.csv')
ratings_df = pd.read_csv('./ratings_without_timestamp.csv')

In [9]:
# movie_df

In [ ]:
from numpy.linalg import svd
matrix = reviewmatrix.values
u, s, vh = svd(matrix, full_matrices=False)